In [1]:
import requests
import pandas as pd
from datetime import date
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_ta
import numpy as np

In [2]:
# Nifty 50 Stocks

df_stocks = pd.read_csv("https://archives.nseindia.com/content/indices/ind_nifty50list.csv")

In [3]:
# Preparing Start Date and End Date for fetching data from yahooFinance.com.

today  = date.today()
enddate = time.mktime(today.timetuple())
enddate = str(int(enddate))


startdate = datetime.now().date().replace(year=2019, month=2, day=17)
stdate=time.mktime(startdate.timetuple())
stdate=str(int(stdate))

In [4]:
# Stock to predict

stock = 'ADANIENT'

In [5]:
# url for historical data 

url = "https://query1.finance.yahoo.com/v7/finance/download/" +stock+ ".NS?period1=" + stdate+ "&period2=" + enddate+ "&interval=1d&events=history&includeAdjustedClose=true"

In [6]:
df = pd.read_csv(url)

In [7]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-02-18,118.300003,119.900002,116.099998,117.500000,116.166290,2569451
1,2019-02-19,118.400002,119.800003,116.800003,118.250000,116.907784,2644152
2,2019-02-20,117.949997,121.099998,116.199997,120.050003,118.687355,3739081
3,2019-02-21,119.800003,121.000000,117.900002,119.400002,118.044731,2698192
4,2019-02-22,120.250000,130.699997,119.800003,128.050003,126.596550,14695085
...,...,...,...,...,...,...,...
986,2023-02-13,1850.000000,1921.699951,1662.250000,1717.650024,1717.650024,12210940
987,2023-02-14,1735.000000,1889.000000,1611.349976,1749.699951,1749.699951,14579030
988,2023-02-15,1780.000000,1824.400024,1750.000000,1779.099976,1779.099976,7636578
989,2023-02-16,1820.000000,1874.949951,1790.000000,1796.599976,1796.599976,5578515


In [8]:
df2 = df.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)

In [9]:
df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [10]:
# Converting the date data type (object --> datetime)

df['Date'] = pd.to_datetime(df['Date'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       991 non-null    datetime64[ns]
 1   Open       991 non-null    float64       
 2   High       991 non-null    float64       
 3   Low        991 non-null    float64       
 4   Close      991 non-null    float64       
 5   Adj Close  991 non-null    float64       
 6   Volume     991 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 54.3 KB


In [12]:
# Setting date column as index

df = df.set_index('Date')

In [13]:
df.index

DatetimeIndex(['2019-02-18', '2019-02-19', '2019-02-20', '2019-02-21',
               '2019-02-22', '2019-02-25', '2019-02-26', '2019-02-27',
               '2019-02-28', '2019-03-01',
               ...
               '2023-02-06', '2023-02-07', '2023-02-08', '2023-02-09',
               '2023-02-10', '2023-02-13', '2023-02-14', '2023-02-15',
               '2023-02-16', '2023-02-17'],
              dtype='datetime64[ns]', name='Date', length=991, freq=None)

# LSTM MODEL 

In [14]:
df1 = df[['Close']]

In [15]:
# Scaling 

from sklearn.preprocessing import MinMaxScaler
MinMax = MinMaxScaler()
df1 = MinMax.fit_transform(df1)

In [16]:
# Splitting dataset into train and test

training_size = int(len(df1)*0.70)
test_size = len(df1)-training_size 
train_data = df1[0:training_size,:]
test_data = df1[training_size :len(df1), :1]

In [17]:
# Convert an array of values into dataset matrix

def create_dataset(dataset, time_step=1):
    X,Y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

In [18]:
X_train, Y_train = create_dataset(train_data, 100)
X_test, Y_test = create_dataset(test_data, 100)

In [19]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [20]:
### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [21]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
10/10 [==============================] - 21s 601ms/step - loss: 0.0166 - val_loss: 0.0334
Epoch 2/100
10/10 [==============================] - 3s 271ms/step - loss: 0.0019 - val_loss: 0.0469
Epoch 3/100
10/10 [==============================] - 3s 278ms/step - loss: 8.2872e-04 - val_loss: 0.0111
Epoch 4/100
10/10 [==============================] - 3s 289ms/step - loss: 4.6933e-04 - val_loss: 0.0140
Epoch 5/100
10/10 [==============================] - 3s 266ms/step - loss: 3.2259e-04 - val_loss: 0.0106
Epoch 6/100
10/10 [==============================] - 3s 261ms/step - loss: 2.6520e-04 - val_loss: 0.0122
Epoch 7/100
10/10 [==============================] - 3s 262ms/step - loss: 2.4932e-04 - val_loss: 0.0122
Epoch 8/100
10/10 [==============================] - 3s 263ms/step - loss: 2.3331e-04 - val_loss: 0.0129
Epoch 9/100
10/10 [==============================] - 3s 260ms/step - loss: 2.2919e-04 - val_loss: 0.0140
Epoch 10/100
10/10 [==============================] - 3s 257ms

10/10 [==============================] - 2s 249ms/step - loss: 1.4846e-04 - val_loss: 0.0275
Epoch 79/100
10/10 [==============================] - 2s 248ms/step - loss: 1.7193e-04 - val_loss: 0.0282
Epoch 80/100
10/10 [==============================] - 2s 246ms/step - loss: 1.5295e-04 - val_loss: 0.0296
Epoch 81/100
10/10 [==============================] - 2s 243ms/step - loss: 1.5203e-04 - val_loss: 0.0239
Epoch 82/100
10/10 [==============================] - 2s 245ms/step - loss: 1.8959e-04 - val_loss: 0.0210
Epoch 83/100
10/10 [==============================] - 2s 246ms/step - loss: 2.0421e-04 - val_loss: 0.0290
Epoch 84/100
10/10 [==============================] - 2s 248ms/step - loss: 2.1227e-04 - val_loss: 0.0273
Epoch 85/100
10/10 [==============================] - 2s 248ms/step - loss: 1.5860e-04 - val_loss: 0.0294
Epoch 86/100
10/10 [==============================] - 2s 248ms/step - loss: 1.7092e-04 - val_loss: 0.0344
Epoch 87/100
10/10 [==============================] - 3s 25

In [24]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

7/7 [==============================] - 0s 60ms/step


In [25]:
train_predict=MinMax.inverse_transform(train_predict)
test_predict=MinMax.inverse_transform(test_predict)

# Validating the Fit 

In [26]:
from sklearn.metrics import mean_squared_error, r2_score

In [27]:
# Train Data RMSE

np.sqrt(mean_squared_error(Y_train,train_predict))

868.5210528775762

In [28]:
# Test Data RMSE

np.sqrt(mean_squared_error(Y_test,test_predict))

2604.5006780196027

In [29]:
# Train Data MSE

mean_squared_error(Y_train,train_predict)

754328.8192915735

In [30]:
# Test Data MSE

mean_squared_error(Y_test,test_predict)

6783423.781804571